In [1]:
%pip install --upgrade --quiet  langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [5]:
import textwrap

from langchain_community.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import AzureChatOpenAI  

import openai_config  

import warnings
warnings.filterwarnings("ignore")

In [6]:
NEO4J_URI = "neo4j://192.168.0.87:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

In [7]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [8]:
llm = AzureChatOpenAI(api_key=openai_config.api_key, api_version=openai_config.api_version, azure_endpoint=openai_config.api_base, azure_deployment=openai_config.deployment_id, temperature = 0)

In [9]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Movie {tagline: STRING, title: STRING,
released: INTEGER} Person {name: STRING, born: INTEGER}
Relationship properties: ACTED_IN {roles: LIST} REVIEWED
{rating: INTEGER, summary: STRING} The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie) (:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]->(:Person)
(:Person)-[:REVIEWED]->(:Movie)


In [10]:
kg.query("MATCH (n) RETURN n LIMIT 10")

[{'n': {'tagline': 'Welcome to the Real World',
   'title': 'The Matrix',
   'released': 1999}},
 {'n': {'born': 1964, 'name': 'Keanu Reeves'}},
 {'n': {'born': 1967, 'name': 'Carrie-Anne Moss'}},
 {'n': {'born': 1961, 'name': 'Laurence Fishburne'}},
 {'n': {'born': 1960, 'name': 'Hugo Weaving'}},
 {'n': {'born': 1967, 'name': 'Lilly Wachowski'}},
 {'n': {'born': 1965, 'name': 'Lana Wachowski'}},
 {'n': {'born': 1952, 'name': 'Joel Silver'}},
 {'n': {'born': 1978, 'name': 'Emil Eifrem'}},
 {'n': {'tagline': 'Free your mind',
   'title': 'The Matrix Reloaded',
   'released': 2003}}]

In [11]:
CYPHER_GENERATION_TEMPLATE = """
# Task:Generate Cypher statement to 
query a graph database.

# Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.

# Schema:
{schema}

# Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# The question is:
{question}"""

In [12]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [13]:
cypherChain = GraphCypherQAChain.from_llm(
    llm,
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [14]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [15]:
prettyCypherChain("How many actors are?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(:Movie)
RETURN COUNT(DISTINCT p)
Full Context:
[{'COUNT(DISTINCT p)': 102}]

> Finished chain.
There are 102 distinct actors.


In [16]:
prettyCypherChain("give me 10 actor names from the db")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(:Movie)
RETURN p.name
LIMIT 10
Full Context:
[{'p.name': 'Emil Eifrem'}, {'p.name': 'Hugo Weaving'}, {'p.name': 'Laurence Fishburne'}, {'p.name': 'Carrie-Anne Moss'}, {'p.name': 'Keanu Reeves'}, {'p.name': 'Hugo Weaving'}, {'p.name': 'Laurence Fishburne'}, {'p.name': 'Carrie-Anne Moss'}, {'p.name': 'Keanu Reeves'}, {'p.name': 'Hugo Weaving'}]

> Finished chain.
The ten actor names from the database are Emil Eifrem, Hugo
Weaving, Laurence Fishburne, Carrie-Anne Moss, and Keanu
Reeves. Please note that some names are repeated.


In [20]:
prettyCypherChain("in which movies acted Hugo Weaving?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Hugo Weaving'})-[:ACTED_IN]->(m:Movie)
RETURN m.title
Full Context:
[{'m.title': 'Cloud Atlas'}, {'m.title': 'V for Vendetta'}, {'m.title': 'The Matrix Revolutions'}, {'m.title': 'The Matrix Reloaded'}, {'m.title': 'The Matrix'}]

> Finished chain.
Hugo Weaving acted in 'Cloud Atlas', 'V for Vendetta', 'The
Matrix Revolutions', 'The Matrix Reloaded', and 'The
Matrix'.


In [18]:
prettyCypherChain("are movie names that contains 'The Matrix' in the database? If yes give me their titles")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
WHERE m.title CONTAINS 'The Matrix'
RETURN m.title
Full Context:
[{'m.title': 'The Matrix'}, {'m.title': 'The Matrix Reloaded'}, {'m.title': 'The Matrix Revolutions'}]

> Finished chain.
Yes, the movie titles that contain 'The Matrix' in the
database are 'The Matrix', 'The Matrix Reloaded', and 'The
Matrix Revolutions'.


In [19]:
prettyCypherChain("give me the cast for the movie 'The Da Vinci Code'")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'The Da Vinci Code'})<-[:ACTED_IN]-(p:Person)
RETURN p.name as Cast
Full Context:
[{'Cast': 'Tom Hanks'}, {'Cast': 'Ian McKellen'}, {'Cast': 'Audrey Tautou'}, {'Cast': 'Paul Bettany'}]

> Finished chain.
The cast for the movie 'The Da Vinci Code' includes Tom
Hanks, Ian McKellen, Audrey Tautou, and Paul Bettany.
